# エラー内容をレスポンスに記載する

In [1]:
from flask import Flask, Response, request

app = Flask(__name__)

@app.route("/try",methods=["POST"])
def try_example():
    json = request.get_json()
    # try以下のコードが通らなかった時にexcept以下が実行
    try:
        # 入力が文字列だとint型に変換できないためエラーが発生
        num = int(json["num"])
        return Response(str(num), mimetype = "text/plain")
    
    except:
        # ステータスコードは２つ目の返り値で指定
        return Response("有効な数字ではありません。",mimetype = "text/plain"), 500
    
app.run(debug=False, host='0.0.0.0', port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.18.0.1 - - [14/Feb/2020 11:33:48] "POST /try HTTP/1.1" 500 -
172.18.0.1 - - [14/Feb/2020 11:34:10] "POST /try HTTP/1.1" 200 -


http://127.0.0.1:5000/try
{"num":3}

{"num":"aaaaa"}

# エラーログを残す

In [2]:
import sys
a = "aggkhbj"
try:
    int(a)
except Exception as e:
    _, error_message, tb=sys.exc_info()
    lineno = tb.tb_lineno
    print(lineno)
    print(str(error_message))

4
invalid literal for int() with base 10: 'aggkhbj'


In [3]:
from flask import Flask, Response, request
import datetime #現在時刻を取得するのに利用
import sys #エラーハンドリングの際に利用

app = Flask(__name__)

@app.route("/errorhand",methods=["POST"])
def errorhand():
    try:
        json = request.get_json()
        num = int(json["num"])
        return Response(str(num),mimetype = "text/plain")
    except Exception as e:
        error_detail = errorlogging()#エラーロギングの関数にエラー内容を渡す
        
        #プログラムのエラーはそのままレスポンスしない
        return "INTERNAL SERVER ERROR", 500

    
def errorlogging():
    fn = "flask_error.log"#ログの出力先を指定
    now = datetime.datetime.now()#現在時刻を取得
    now = now.strftime("%Y/%m/%d %H:%M:%S")#時刻のフォーマットを変更
    _, error_message, tb=sys.exc_info()#エラー情報を取得
    lineno = tb.tb_lineno#エラーが発生したプログラムの行数を取得
    error_detail = {
                            "timestamp":now,
                            "program":"error.ipynb",
                            "line":lineno,
                            "message": str(error_message)
                           }
    error_detail = str(error_detail)#一行でファイル出力するためにstr型に変換
    #エラーの詳細を書き出す
    with open(fn,"a") as f:
        f.write(error_detail+"\n")
    return error_detail

        
app.run(host='0.0.0.0', port=5000) 

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.18.0.1 - - [14/Feb/2020 11:40:51] "POST /errorhand HTTP/1.1" 200 -
172.18.0.1 - - [14/Feb/2020 11:43:10] "POST /errorhand HTTP/1.1" 500 -
172.18.0.1 - - [14/Feb/2020 11:43:55] "POST /errorhand HTTP/1.1" 200 -
172.18.0.1 - - [14/Feb/2020 11:44:06] "POST /errorhand HTTP/1.1" 500 -


http://127.0.0.1:5000/errorhand
{"num":3}